In [2]:
import numpy as np
import jax.random
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

import pennylane as qml
from pennylane import numpy as pnp
from pennylane.optimize import NesterovMomentumOptimizer
from shapely.geometry import Polygon, Point

from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.ioff()

import multiprocessing as mp
import time
import pandas as pd
from datetime import datetime
import os
import pytz
import ast
from pypdf import PdfMerger

import warnings
# warnings.filterwarnings('ignore')

In [347]:
# Running parameters
num_iters = 1    # Number of training iterations
num_runs = 1
with_cv = False
with_val = False

# Circuit and optimization parameters
nqubits = 4         # Num qubits, min 4, always 2**num_layers qubits
with_bias = False    # Add a bias to the output of the quantum circuit
random = True
optimizer = "Adam"  # "Adam", "Nesterov", "GradientDescent"
loss_type = "cross-entropy" # "projectors", "cross-entropy"

# Data hyper-parameters
batch_size = 100     # batch training size
train_size = 100      # Total ground states that will be used for training
val_size = 1000      # Total gound states with training + validation
cv_ratios = [0.3, 0.2, 0.2, 0.2, 0.1]

# How the training data is generated
uniform_train = True    # True - Uniform, False - Balanced
epsilon_train = True   # True - epsilon, False - no epsilon
uniform_val = True
epsilon_val = False

# Multiprocess hyperparameters. recommended: (num_data, num_cpus) -> (20, 10), (100, 20-30), (200, 40-50) (1000, 50-70)
num_cpus_batch = 0 # 20
num_cpus_train = 0 # 20
num_cpus_val = 0 # 60

# Tweak training hyper-parameters
max_weight_init = 2*np.pi  # weight_init goes from 0 to this number. Max = 2*np.pi. Other options = 0.01
stepsize = 0.01         # stepsize of the gradient descent.

# Constant definitions
layers = int(np.log2(nqubits))
nweights = 30*(layers-1) + 15

In [4]:
def X(i):
    return qml.PauliX(i)

def Z(i):
    return qml.PauliZ(i)

# Gound states

In [276]:
def ground_state(j1, j2):
    
    hamiltonian = 0
    for i in range(nqubits):
        hamiltonian += Z(i)
        hamiltonian -= j1 * X(i) @ X((i+1)%nqubits)
        hamiltonian -= j2 * X((i-1)%nqubits) @ Z(i) @ X((i+1)%nqubits)
    
    _, eigvecs = np.linalg.eigh(qml.matrix(hamiltonian))
    
    return eigvecs[:,0]

In [283]:
# Define coordinates of the points of each region# Definir las coordenadas de los puntos de cada región
region01_coords = np.array([(-2, 1), (2, 1), (4, 3), (4, 4), (-4, 4), (-4, 3)])    # Class 0
region02_coords = np.array([(-3, -4), (0, -1), (3, -4)])                           # Class 0
region1_coords = np.array([(0, -1), (3, -4), (4, -4), (4, 3)])                     # Class 1
region2_coords = np.array([(0, -1), (-3, -4), (-4, -4), (-4, 3)])                  # Class 2
region3_coords = np.array([(-2, 1), (2, 1), (0, -1)])                              # Class 3

e = 0.1
# Define coordinates of the points of each region far from the borders
region01e_coords = np.array([(-2+(np.sqrt(2)-1)*e, 1+e), (2-(np.sqrt(2)-1)*e, 1+e), (4, 3+np.sqrt(2)*e), (4, 4), (-4, 4), (-4, 3+np.sqrt(2)*e)])    # Class 0 with epsilon
region02e_coords = np.array([(-3+np.sqrt(2)*e, -4), (0, -1-np.sqrt(2)*e), (3-np.sqrt(2)*e, -4)])                                                    # Class 0 with epsilon
region1e_coords = np.array([(0+np.sqrt(2)*e, -1), (3+np.sqrt(2)*e, -4), (4, -4), (4, 3-np.sqrt(2)*e)])                                              # Class 1 with epsilon
region2e_coords = np.array([(0-np.sqrt(2)*e, -1), (-3-np.sqrt(2)*e, -4), (-4, -4), (-4, 3-np.sqrt(2)*e)])                                           # Class 2 with epsilon
region3e_coords = np.array([(-2+e/np.tan(np.pi/8), 1-e), (2-e/np.tan(np.pi/8), 1-e), (0, -1+np.sqrt(2)*e)])                                         # Class 3 with epsilon


def labeling(x, y):

    # Create Polygons for each region
    region01_poly = Polygon(region01_coords)
    region02_poly = Polygon(region02_coords)
    region1_poly = Polygon(region1_coords)
    region2_poly = Polygon(region2_coords)
    region3_poly = Polygon(region3_coords)
    
    p = Point(x, y)
    if region01_poly.contains(p):
        return 0
    elif region02_poly.contains(p):
        return 0
    elif region1_poly.contains(p):
        return 1
    elif region2_poly.contains(p):
        return 2
    elif region3_poly.contains(p):
        return 3
    else:
        return None # if the point is not in any region
    

def labeling_epsilon(x, y):
    
    # Create Polygons for each region
    region01e_poly = Polygon(region01e_coords)
    region02e_poly = Polygon(region02e_coords)
    region1e_poly = Polygon(region1e_coords)
    region2e_poly = Polygon(region2e_coords)
    region3e_poly = Polygon(region3e_coords)
    
    
    p = Point(x, y)
    if region01e_poly.contains(p):
        return 0
    elif region02e_poly.contains(p):
        return 0
    elif region1e_poly.contains(p):
        return 1
    elif region2e_poly.contains(p):
        return 2
    elif region3e_poly.contains(p):
        return 3
    else:
        return None # if the point is not in any region

In [289]:
# Generate ground states
def generate_gs(num_points, uniform, epsilon, num_cpus):

    if random:
        rng = np.random
    else:
        rng = np.random.RandomState(0)
        

    if uniform:
        if epsilon:
            j_list = []
            num = 0
            while num < num_points:
                j = rng.uniform(-4, 4, 2)
                l = labeling_epsilon(j[0], j[1])

                if l in [0,1,2,3]:
                    num += 1
                    j_list.append(j)

            j_list = np.array(j_list)
            
        else:
            j_list = rng.uniform(-4, 4, (num_points,2))
    
    
    else: # Then it's balanced
        npoints_class = num_points//4
        num_points = 4*npoints_class
        
        npoints_02 = npoints_class//2
        npoints_01 = npoints_class - npoints_02
        
        j_list = []
        num_0, num_1, num_2, num_3 = 0, 0, 0, 0
        num_01, num_02 = 0, 0
        
        while num_0 != npoints_class or num_1 != npoints_class or num_2 != npoints_class or num_3 != npoints_class:
            j = rng.uniform(-4, 4, 2)
            l = labeling_epsilon(j[0], j[1]) if epsilon else labeling(j[0], j[1])

            if l==0 and num_0 < npoints_class:
                
                p = Point(j[0], j[1])
                if Polygon(region01_coords).contains(p) and num_01 < npoints_01:
                    num_01 += 1
                    num_0 += 1
                    j_list.append(j)
                elif Polygon(region02_coords).contains(p) and num_02 < npoints_02:
                    num_02 += 1
                    num_0 += 1
                    j_list.append(j)
                
            elif l==1 and num_1 < npoints_class:
                num_1 += 1
                j_list.append(j)
            elif l==2 and num_2 < npoints_class:
                num_2 += 1
                j_list.append(j)
            elif l==3 and num_3 < npoints_class:
                num_3 += 1
                j_list.append(j)

        j_list = np.array(j_list)
    
    
    if num_cpus == 0:
        gs_list = []
        labels_list = []

        for i in range(num_points):
            gs_list.append(ground_state(j_list[i,0], j_list[i,1]))
            labels_list.append(labeling(j_list[i,0], j_list[i,1]))
            
    else:
        args = [[j_list[i,0], j_list[i,1]] for i in range(num_points)]
        
        with mp.Pool(num_cpus) as pool:
            gs_list = pool.starmap(ground_state, args)
        
        with mp.Pool(num_cpus) as pool:
            labels_list = pool.starmap(labeling, args)
            
        
    gs_list = np.array(gs_list)
    labels_list = np.array(labels_list)

    return gs_list, labels_list, j_list

In [238]:
if random:
    rng = np.random
else:
    rng = np.random.RandomState(0)

if uniform_train:
    if epsilon_train:
        j_list = []
        num = 0
        while num < train_size:
            j = rng.uniform(-4, 4, 2)
            l = labeling_epsilon(j[0], j[1])

            if l in [0,1,2,3]:
                num += 1
                j_list.append(j)

        j_list = np.array(j_list)

    else:
        j_list = rng.uniform(-4, 4, (train_size,2))


else: # Then it's balanced
    npoints_class = train_size//4
    train_size = 4*npoints_class

    npoints_02 = npoints_class//2
    npoints_01 = npoints_class - npoints_02

    j_list = []
    num_0, num_1, num_2, num_3 = 0, 0, 0, 0
    num_01, num_02 = 0, 0

    while num_0 != npoints_class or num_1 != npoints_class or num_2 != npoints_class or num_3 != npoints_class:
        j = rng.uniform(-4, 4, 2)
        l = labeling_epsilon(j[0], j[1]) if epsilon else labeling(j[0], j[1])

        if l==0 and num_0 < npoints_class:

            p = Point(j[0], j[1])
            if Polygon(region01_coords).contains(p) and num_01 < npoints_01:
                num_01 += 1
                num_0 += 1
                j_list.append(j)
            elif Polygon(region02_coords).contains(p) and num_02 < npoints_02:
                num_02 += 1
                num_0 += 1
                j_list.append(j)

        elif l==1 and num_1 < npoints_class:
            num_1 += 1
            j_list.append(j)
        elif l==2 and num_2 < npoints_class:
            num_2 += 1
            j_list.append(j)
        elif l==3 and num_3 < npoints_class:
            num_3 += 1
            j_list.append(j)

    j_list = np.array(j_list)
    

print(j_list.shape)
print(j_list[:,0])
print(j_list[:,1])

(100, 2)
[ 2.12611569 -3.37639534  2.45000649  0.09979407  3.96435009 -1.07803785
 -2.59183817  3.28342971 -2.80694549  1.59064832  1.53908378  1.88766239
 -3.39527576 -2.12367675  1.41335459  1.38396754  1.62973072  0.54196749
 -2.79599334 -2.67731975 -0.29237662  2.7802531  -0.57809252  2.13435367
 -2.16713796 -3.21287231 -3.47191386 -0.92209651 -3.34452764  2.96391999
  2.33096511 -0.39059666 -2.3028582  -0.37293537 -0.10968401 -3.37685385
 -0.49235625  2.25173347 -1.1850162  -0.20594142  2.42209747  0.43669682
  3.84877667  3.94578742 -2.27634508 -1.05295999  2.80995193 -2.5363149
  1.14819627 -0.17520086 -2.88125005 -0.3573416   1.95082736 -2.7206305
 -2.3312757   2.0142146   1.06568167 -2.52420489 -2.9006123   1.13783175
  1.91623813  0.29927629  0.09204716  1.95373268  2.2874722   3.66529771
  0.90457167 -3.95141965 -2.54569406  0.10459099 -0.13983748 -1.93353052
 -3.0997225   0.51913604  2.39201067  2.37517378 -1.70429535 -3.17466186
  3.23359118  3.0180807  -2.82880352  0.6265

# CNN

In [8]:
def convolutional_layer(q1, q2, weights):
    qml.U3(wires=q1, theta=weights[0], phi=weights[1], delta=weights[2])
    qml.U3(wires=q1, theta=weights[3], phi=weights[4], delta=weights[5])
    qml.CNOT(wires=[q2, q1])
    qml.RZ(wires=q1, phi=weights[6])
    qml.RY(wires=q2, phi=weights[7])
    qml.CNOT(wires=[q1, q2])
    qml.RY(wires=q2, phi=weights[8])
    qml.CNOT(wires=[q2, q1])
    qml.U3(wires=q1, theta=weights[9], phi=weights[10], delta=weights[11])
    qml.U3(wires=q1, theta=weights[12], phi=weights[13], delta=weights[14])

def pooling_layer(q1, q2, weights):
    qml.U3(wires=q1, theta=weights[0], phi=weights[1], delta=weights[2])
    qml.U3(wires=q1, theta=weights[3], phi=weights[4], delta=weights[5])
    qml.CNOT(wires=[q2, q1])
    qml.RZ(wires=q1, phi=weights[6])
    qml.RY(wires=q2, phi=weights[7])
    qml.CNOT(wires=[q1, q2])
    qml.RY(wires=q2, phi=weights[8])
    qml.CNOT(wires=[q2, q1])
    qml.U3(wires=q1, theta=weights[9], phi=weights[10], delta=weights[11])
    qml.U3(wires=q1, theta=weights[12], phi=weights[13], delta=weights[14])

In [177]:
def cnn_circuit(weights, state_ini):
    
    qubits = list(range(nqubits))
    
    qml.QubitStateVector(state_ini, wires=qubits)

    for j in range(layers-1):
        
        len_qubits = len(qubits)
        
        for i in range(len_qubits//2):
            convolutional_layer(qubits[2*i], qubits[(2*i+1)%len_qubits], weights[15*2*j:15*(2*j+1)])
        
        for i in range(len_qubits//2):
            convolutional_layer(qubits[2*i+1], qubits[(2*i+2)%len_qubits], weights[15*2*j:15*(2*j+1)])
            
        for i in range(len_qubits//2):
            pooling_layer(qubits[2*i], qubits[(2*i+1)%len_qubits], weights[15*(2*j+1):15*(2*j+2)])

        qub = []
        for i in range(len_qubits):
            if i%2 == 1:
                qub.append(qubits[i])
                
        qubits = qub
    
    convolutional_layer(qubits[0], qubits[1], weights[15*(2*layers-2):15*(2*layers-1)])
    
    return qml.expval(Z(qubits[0])), qml.expval(Z(qubits[1])), qml.expval(Z(qubits[0]) @ Z(qubits[1]))

# dev_draw = qml.device("qiskit.aer", wires=nqubits)
dev = qml.device("default.qubit", wires=nqubits)

# cnn_draw = qml.QNode(cnn_circuit, dev_draw)
cnn_circuit = qml.QNode(cnn_circuit, dev, interface="jax", diff_method="best")


def cnn(weights, state_ini):
    z0, z1, zz01 = cnn_circuit(weights, state_ini)

    proj_00 = (1+zz01+z0+z1)/4
    proj_01 = (1-zz01-z0+z1)/4
    proj_10 = (1-zz01+z0-z1)/4
    proj_11 = (1+zz01-z0-z1)/4

    return jnp.array([proj_00, proj_01, proj_10, proj_11])

In [304]:
@jax.jit
def variational_classifier(weights, bias, state_ini):
    return cnn(weights, state_ini) + bias

In [11]:
# draw circuit

# weights = np.random.uniform(0, 2*np.pi, nweights)
# drawer = qml.draw(cnn_circuit)
# print(drawer(weights, gs_list[0]))

# z0, z1, zz01 = cnn_draw(gs_list[0], weights)
# print(z0, z1, zz01)
# dev_draw._circuit.draw(output="mpl")

# Loss and accuracy

In [179]:
@jax.jit
def single_loss(weights, bias, ground_state, label):
    
    proj = variational_classifier(weights, bias, ground_state)

    if loss_type == "projectors":
            cost = proj[label]

    elif loss_type == "cross-entropy":
        
            # if isinstance(proj[label], pnp._np.numpy_boxes.ArrayBox):
            #     cost = -log2_arraybox(proj[label])
            # else:
            #     cost = -np.log2(proj[label])
                
        cost = -jnp.log2(proj[label])
    
    return cost

@jax.jit
def loss(weights, bias, ground_states, labels, num_cpus):
    
    # cost=0
    # for j in range(len(labels)):
    #     cost += single_loss(weights, bias, ground_states[j], labels[j])
        
    costs = jax.vmap(single_loss, in_axes=[None, None, 0, 0])(weights, bias, ground_states, labels)
    cost = costs.sum()
    
    return cost/len(labels)

@jax.jit
def grad_loss(weight, bias, ground_states, labels, _):
    w, b = jax.grad(loss, argnums=[0,1])(weight, bias, ground_states, labels, None)
    return w, b, None, None, None # maybe same array shape as input? jnp.zeros?

In [349]:
@jax.jit
def single_pred(weights, bias, ground_state):
    
    projectors = variational_classifier(weights, bias, ground_state)
    
    if loss_type == "projectors":
        pred = np.argmin(projectors)
    elif loss_type == "cross-entropy":
        pred = np.argmax(projectors)

    return pred

@partial(jax.jit, static_argnums=3)
def pred(weights, bias, ground_states, num_cpus):
    
    if num_cpus == 0:
        predictions = []
        
        for j in range(len(ground_states)):
            predictions.append(single_pred(weights, bias, ground_states[j]))
        
        predictions = np.array(predictions)
        
    else:
        predictions = jax.vmap(single_pred, in_axes=[None, None, 0])(weights, bias, ground_states)
            
    return predictions

@jax.jit
def acc(predictions, labels):
    return sum(predictions==labels)*100/len(labels)

# Processing Data

In [342]:
def save_multi_image(filename):
    pp = PdfPages(filename)
    fig_nums = plt.get_fignums()
    figs = [plt.figure(n) for n in fig_nums]
    for fig in figs:
        fig.savefig(pp, format='pdf')
    pp.close()

def close_all_figures():
    fig_nums = plt.get_fignums()
    for n in fig_nums:
        plt.figure(n)
        plt.close()

        
def save_plots(time_now,
               folder_name,
               file_name,
               plot_run,
               it_max,
               acc_train,
               acc_val,
               losses,
               pred_train,
               pred_val,
               j_train,
               j_val
              ):


    fig, axis = plt.subplots(1,3)
    fig.set_figheight(6.5)
    fig.set_figwidth(20)
    fig.tight_layout(pad=2, w_pad=3.5)

    # ---------------------------------------------------------------------- #
    # -------------------- Loss and accuracy figure ------------------------ #
    # ---------------------------------------------------------------------- #

    iterations = range(1, num_iters+1)

    color1 = 'darkred'
    axis[0].set_xlabel('Iterations')
    axis[0].set_ylabel('Accuracy %', color=color1)
    axis[0].plot(iterations, acc_train, label="Training", color=color1)
    axis[0].plot(iterations, acc_val, '-.', label="Validation", color=color1)
    axis[0].tick_params(axis='y', labelcolor=color1)
    axis[0].set_ylim(0,100)

    ax2 = axis[0].twinx()  # instantiate a second axes that shares the same x-axis

    color2 = 'darkblue'
    ax2.set_ylabel('Loss', color=color2)  # we already handled the x-label with axis[0]
    ax2.plot(iterations, losses, label="Loss", color=color2)
    ax2.tick_params(axis='y', labelcolor=color2)
    # ax2.set_ylim(bottom=0)

    # fig.tight_layout()  # otherwise the right y-label is slightly clipped
    # plt.legend()
    axis[0].set_title(f"Accuracy and Loss - Run {plot_run}")


    # ---------------------------------------------------------------------- #
    # ---------------------------- Max iter -------------------------------- #
    # ---------------------------------------------------------------------- #

    plot_iter = it_max

    # define regions coordinates
    x01, y01 = region01_coords[:,0], region01_coords[:,1]
    x02, y02 = region02_coords[:,0], region02_coords[:,1]
    x1, y1 = region1_coords[:,0], region1_coords[:,1]
    x2, y2 = region2_coords[:,0], region2_coords[:,1]
    x3, y3 = region3_coords[:,0], region3_coords[:,1]

    # put the regions into the plot
    axis[1].fill(x01, y01, facecolor='lightskyblue')    # class 0
    axis[1].fill(x02, y02, facecolor='lightskyblue')    # class 0
    axis[1].fill(x1, y1, facecolor='sandybrown')        # class 1
    axis[1].fill(x2, y2, facecolor='salmon')            # class 2
    axis[1].fill(x3, y3, facecolor='lightgreen')        # class 3

    pred_train_plot = np.array(pred_train[plot_iter])
    pred_val_plot = np.array(pred_val[plot_iter])

    colors = ["b", "orange", "r", "g"]

    # plot datapoints
    for i in range(4):
        axis[1].scatter(
            j_train[:, 0][pred_train_plot==i],
            j_train[:, 1][pred_train_plot==i],
            c=colors[i],
            marker="o",
            edgecolors="k",
            label=f"class {i+1} train",
        )
        if with_val:
            axis[1].scatter(
                j_val[:, 0][pred_val_plot==i],
                j_val[:, 1][pred_val_plot==i],
                c=colors[i],
                marker="^",
                edgecolors="k",
                label=f"class {i+1} validation",
            )


    # plt.legend()
    axis[1].set_title(f"Max iteration ({acc_train[plot_iter]:.0f}%/{acc_val[plot_iter]:.0f}%)")


    # ---------------------------------------------------------------------- #
    # ---------------------------- Last iter ------------------------------- #
    # ---------------------------------------------------------------------- #


    plot_iter = num_iters-1

    # define regions coordinates
    x01, y01 = region01_coords[:,0], region01_coords[:,1]
    x02, y02 = region02_coords[:,0], region02_coords[:,1]
    x1, y1 = region1_coords[:,0], region1_coords[:,1]
    x2, y2 = region2_coords[:,0], region2_coords[:,1]
    x3, y3 = region3_coords[:,0], region3_coords[:,1]

    # put the regions into the plot
    axis[2].fill(x01, y01, facecolor='lightskyblue')    # class 0
    axis[2].fill(x02, y02, facecolor='lightskyblue')    # class 0
    axis[2].fill(x1, y1, facecolor='sandybrown')        # class 1
    axis[2].fill(x2, y2, facecolor='salmon')            # class 2
    axis[2].fill(x3, y3, facecolor='lightgreen')        # class 3

    pred_train_plot = np.array(pred_train[plot_iter])
    pred_val_plot = np.array(pred_val[plot_iter])

    colors = ["b", "orange", "r", "g"]

    # plot datapoints
    for i in range(4):
        axis[2].scatter(
            j_train[:, 0][pred_train_plot==i],
            j_train[:, 1][pred_train_plot==i],
            c=colors[i],
            marker="o",
            edgecolors="k",
            label=f"class {i+1} train",
        )
        if with_val:
            axis[2].scatter(
                j_val[:, 0][pred_val_plot==i],
                j_val[:, 1][pred_val_plot==i],
                c=colors[i],
                marker="^",
                edgecolors="k",
                label=f"class {i+1} validation",
            )


    # plt.legend()
    axis[2].set_title(f"Last iteration ({acc_train[plot_iter]:.0f}%/{acc_val[plot_iter]:.0f}%)")

    # ---------------------------------------------------------------------- #
    # --------------------------- Save plots ------------------------------- #
    # ---------------------------------------------------------------------- #

    plots_pdf_name = f"{folder_name}/{time_now} - Plots - {file_name}.pdf"
    
    
    # If the file doesn't exist we save it. If it does, we merge it.
    if not os.path.isfile(plots_pdf_name):
        save_multi_image(plots_pdf_name)
    
    else:
        save_multi_image(plots_pdf_name + "2")
        # Merge the new plot with the rest and delete the last file
        merger = PdfMerger()
        merger.append(plots_pdf_name)
        merger.append(plots_pdf_name + "2")
        merger.write(plots_pdf_name)
        merger.close()
        os.remove(plots_pdf_name + "2")
    
    close_all_figures()

In [343]:
def save_hyperparameters(time_now, folder_name, file_name):
    
    # --------------- Hyperparameters -----------------#
    hyperparameters = {}
    hyperparameters["num_iters"] = [num_iters]
    hyperparameters["num_runs"] = [num_runs]
    hyperparameters["with_cv"] = [with_cv]
    hyperparameters["with_val"] = [with_val]
    hyperparameters["nqubits"] = [nqubits]
    hyperparameters["with_bias"] = [with_bias]
    hyperparameters["random"] = [random]
    hyperparameters["optimizer"] = [optimizer]
    hyperparameters["loss_type"] = [loss_type]
    hyperparameters["batch_size"] = [batch_size]
    hyperparameters["train_size"] = [train_size]
    hyperparameters["val_size"] = [val_size]
    hyperparameters["cv_ratios"] = [cv_ratios]
    hyperparameters["uniform_train"] = [uniform_train]
    hyperparameters["epsilon_train"] = [epsilon_train]
    hyperparameters["uniform_val"] = [uniform_val]
    hyperparameters["epsilon_val"] = [epsilon_val]
    hyperparameters["num_cpus_batch"] = [num_cpus_batch]
    hyperparameters["num_cpus_train"] = [num_cpus_train]
    hyperparameters["num_cpus_val"] = [num_cpus_val]
    hyperparameters["max_weight_init"] = [max_weight_init]
    hyperparameters["stepsize"] = [stepsize]

    hyperparameters = pd.DataFrame(hyperparameters)

    hyperparameters_file_name = f"{folder_name}/{time_now} - Hyperparameters{file_name}.csv"
    hyperparameters.to_csv(hyperparameters_file_name, index=False)

In [344]:
def save_data(time_now,
                folder_name,
                run,
                weights,
                bias,
                losses,
                j_train,
                j_val,
                pred_train,
                pred_val,
                acc_train,
                acc_val,
                run_time,
                cv
               ):
    
    if cv:
        file_name = f"CV"# - {train_size} Train - {val_size} Val"
    else:
        file_name = f"NCV"# - {train_size} Train - {val_size} Val"
        
    # -------------------- Total Data -------------------- #
    data = {}
    data["run"] = run
    
    it_max = np.argmax(np.array(acc_train))
    acc_train_max = acc_train[it_max]
    acc_train_last = acc_train[num_iters-1]
    acc_val_max = acc_val[it_max]
    acc_val_last = acc_val[num_iters-1]
    
    data["it_max"] = it_max
    data["acc_train_max"] = acc_train_max
    data["acc_train_last"] = acc_train_last
    data["acc_val_max"] = acc_val_max
    data["acc_val_last"] = acc_val_last
    data["run_time"] = run_time
    
    data["weights"] = [weights]
    data["bias"] = [bias]
    data["losses"] = [losses]
    data["j_train"] = [j_train]
    data["j_val"] = [j_val]
    data["pred_train"] = [pred_train]
    data["pred_val"] = [pred_val]
    data["acc_train"] = [acc_train]
    data["acc_val"] = [acc_val]
    

    data = pd.DataFrame(data)
    
    data_file_name = f"{folder_name}/{time_now} - Data - {file_name}.csv"
    data.to_csv(data_file_name, mode='a', index=False, header=not os.path.exists(data_file_name))
    
    
    # ------------------- Results ------------------- #
    
    read_data = pd.read_csv(data_file_name,
                     usecols=["it_max",
                              "acc_train_max",
                              "acc_val_max",
                              "acc_train",
                              "acc_val"],
                     converters={"acc_train":ast.literal_eval,
                                 "acc_val":ast.literal_eval})
    
    total_it_max = read_data["it_max"]
    total_acc_train_max = read_data["acc_train_max"]
    total_acc_val_max = read_data["acc_val_max"]
    total_acc_train = read_data["acc_train"].tolist()
    total_acc_val = read_data["acc_val"].tolist()
    
    best_run_max = total_acc_train_max.argmax()
    best_it_max = total_it_max[best_run_max]
    avg_acc_train_max = total_acc_train_max.mean()
    avg_acc_val_max = total_acc_val_max.mean()
    
    best_run_last = np.argmax(np.array(total_acc_train)[:,num_iters-1])
    avg_acc_train_last = np.mean(np.array(total_acc_train)[:,num_iters-1])
    avg_acc_val_last = np.mean(np.array(total_acc_val)[:,num_iters-1])

    
    results = {}
    results["num_runs"] = [run+1]
    results["best_run_max"] = [best_run_max]
    results["best_run_last"] = [best_run_last]
    results["best_it_max"] = [best_it_max]
    results["best_it_last"] = [num_iters-1]
    results["best_acc_train_max"] = [total_acc_train[best_run_max][best_it_max]]
    results["best_acc_train_last"] = [total_acc_train[best_run_max][num_iters-1]]
    results["best_acc_val_max"] = [total_acc_val[best_run_max][best_it_max]]
    results["best_acc_val_last"] = [total_acc_val[best_run_max][num_iters-1]]
    results["avg_acc_train_max"] = [avg_acc_train_max]
    results["avg_acc_train_last"] = [avg_acc_train_last]
    results["avg_acc_val_max"] = [avg_acc_val_max]
    results["avg_acc_val_last"] = [avg_acc_val_last]

    results = pd.DataFrame(results)

    results_file_name = f"{folder_name}/{time_now} - Results - {file_name}.csv"
    results.to_csv(results_file_name, index=False)
    
    
    # ------------------- Plots ------------------- #
    save_plots(time_now,
               folder_name,
               file_name,
               run,
               it_max,
               acc_train,
               acc_val,
               losses,
               pred_train,
               pred_val,
               j_train,
               j_val
              )
    
    if cv:
        cv_str = "True "
    else:
        cv_str = "False"
        
    print(
        f" {cv_str} |"
        f" {run:3d} |"
        f" {it_max:4d}/{num_iters-1:4d} |"
        f"  {acc_train[it_max]:0.0f}/{acc_train[num_iters-1]:0.0f}  |"
        f" {acc_val[it_max]:0.0f}/{acc_val[num_iters-1]:0.0f} |"
        f" {run_time:0.0f}"
    )
    
    

# Training

In [345]:
def train_qcnn(gs_train, gs_val, labels_train, labels_val, cv):
        
    if random:
        rng = np.random
    else:
        rng = np.random.RandomState(0)
        
    # weights and bias initialization
    #weights_init = jax.random.uniform(0, max_weight_init, nweights)
    key = jax.random.PRNGKey(0)
    weights_init = jax.random.uniform(key, [nweights], minval=0, maxval=max_weight_init)
    bias_init = jnp.array([0.0]*4)

    # choose variational classifier
    if optimizer == "Nesterov":
        opt = NesterovMomentumOptimizer(stepsize)
    elif optimizer == "Adam":
        opt = qml.AdamOptimizer(stepsize=stepsize, beta1=0.9, beta2=0.99, eps=1e-08)
    elif optimizer == "GradientDescent":
        opt = qml.GradientDescentOptimizer(stepsize)

    #Initiaize variables
    weights = []
    bias = []
    losses = []
    pred_train_arr = []
    pred_val_arr = []
    acc_train_arr = []
    acc_val_arr = []

    w = weights_init
    b = bias_init
    
    for it in range(num_iters):
        
        if cv:
            cv_size_batches, cv_limits_it = [], []
            limit_iter, size_batch = 0, 0
            
            for i, ratio in enumerate(cv_ratios):
                
                if i < len(cv_ratios)-1:
                    limit_iter += round(ratio*num_iters)
                    size_batch += round(ratio*len(labels_train))
                else:
                    limit_iter = num_iters
                    size_batch = len(labels_train)
                
                cv_limits_it.append(limit_iter)
                cv_size_batches.append(size_batch)
            
            index_size_batch = np.argmax(it < np.array(cv_limits_it)) # This gives you the first occurrence where the condition is met
            cv_size_batch = cv_size_batches[index_size_batch]
            
            gs_train_batch = gs_train[:cv_size_batch]
            labels_train_batch = labels_train[:cv_size_batch]
        
        else:
            batch_index = np.random.default_rng().choice(len(labels_train), size=batch_size, replace=False)
            
            gs_train_batch = gs_train[batch_index]
            labels_train_batch = labels_train[batch_index]

            
        # Update the weights by one optimizer step
        if num_cpus_batch == 0:
            w, b, _, _, _ = opt.step(loss, w, b, gs_train_batch, labels_train_batch, 0)

        else:
            w, b, _, _, _ = opt.step(loss, w, b, gs_train_batch, labels_train_batch, num_cpus_batch, grad_fn=grad_loss)
            
#             args = [[single_loss, w, b, gs_train_batch[j], labels_train_batch[j]] for j in range(len(labels_train_batch))]

#             with mp.Pool(num_cpus_batch) as pool:
#                 w, b, _, _ = zip(*pool.starmap(opt.step, args))

#             w = sum(w)/len(w)
#             b = sum(b)/len(b)

        weights.append(w)
        bias.append(b)

        # Compute predictions and accuracy on train and validation set
        pred_train = pred(w, b, gs_train, num_cpus_train)
        if with_val:
            pred_val = pred(w, b, gs_val, num_cpus_val) if len(labels_val) > 0 else None
        else:
            pred_val = np.array([0]*len(labels_val))
        
        acc_train = acc(pred_train, labels_train)
        if with_val:
            acc_val = acc(pred_val, labels_val) if len(labels_val) > 0 else 0
        else:
            acc_val = 0
        
        # Save prediction for later plotting
        pred_train_arr.append(pred_train)
        pred_val_arr.append(pred_val)
        acc_train_arr.append(acc_train)
        acc_val_arr.append(acc_val)

        l = loss(w, b, gs_train, labels_train, num_cpus_train)
        losses.append(l)
    
    return weights, bias, losses, pred_train_arr, pred_val_arr, acc_train_arr, acc_val_arr

In [350]:
# with jax.profiler.trace("/tmp/jax-trace", create_perfetto_link=True):

time_now = datetime.now(pytz.timezone('Europe/Andorra')).strftime("%Y-%m-%d %H-%M-%S")

folder_name = f"Results/{nqubits}q - {num_iters:} iters/"
if not os.path.isdir(f'{folder_name}'):
    os.makedirs(f'{folder_name}')

save_hyperparameters(time_now, folder_name, file_name="")


print("Max train / Last run")
print("---------------------------------------------------")
print("  CV   | Run |   Iter    |Acc train|Acc val| Time  ")
print("---------------------------------------------------")


for run in range (num_runs):

    # -------------------------------------------------------------- #
    # ------------------- Generate ground states ------------------- #
    # -------------------------------------------------------------- #

    gs_train, labels_train, j_train = generate_gs(train_size, uniform_train, epsilon_train, num_cpus_train)
    gs_val, labels_val, j_val = generate_gs(val_size, uniform_val, epsilon_val, num_cpus_val)

    # ------------------------------------------------------ #
    # ------------------- Train the QCNN ------------------- #
    # ------------------------------------------------------ #

    start_time = time.time()

    weights, \
    bias, \
    losses, \
    pred_train_arr, \
    pred_val_arr, \
    acc_train_arr, \
    acc_val_arr = train_qcnn(gs_train,
                             gs_val,
                             labels_train,
                             labels_val,
                             cv=False
                            )

    run_time = time.time() - start_time


    # --------------------------------------------------------- #
    # ------------------- Save calculations ------------------- #
    # --------------------------------------------------------- #

    save_data(time_now,
              folder_name,
              run,
              weights,
              bias,
              losses,
              j_train,
              j_val,
              pred_train_arr,
              pred_val_arr,
              acc_train_arr,
              acc_val_arr,
              run_time,
              cv=False
             )


    # ---------------------------------------------------------------------- #
    # ------------------------ QCNN with Curriculum ------------------------ #
    # ---------------------------------------------------------------------- #
    if with_cv:
        # --------------------------------------------------------------------------------- #
        # ------------------------ Sort training gs by their score ------------------------ #
        # --------------------------------------------------------------------------------- #

        score_it = num_iters-1 # num_iters-1 or it_max
        scores = [single_loss(weights[score_it], bias[score_it], gs_train[i], labels_train[i]) for i in range(len(labels_train))]

        table = {}
        table["gs_train"] = gs_train.tolist()
        table["labels_train"] = labels_train
        table["j_train"] = j_train.tolist()
        table["scores"] = scores

        table = pd.DataFrame(table)
        table.sort_values(by=["scores"], inplace=True)

        cv_gs_train = np.array(list(table["gs_train"]))
        cv_labels_train = np.array(list(table["labels_train"]))
        cv_j_train = np.array(list(table["j_train"]))


        # ---------------------------------------------------------------------------- #
        # ------------------------ Train QCNN with Curriculum ------------------------ #
        # ---------------------------------------------------------------------------- #

        start_time = time.time()

        weights, \
        bias, \
        losses, \
        pred_train_arr, \
        pred_val_arr, \
        acc_train_arr, \
        acc_val_arr = train_qcnn(cv_gs_train,
                                 gs_val,
                                 cv_labels_train,
                                 labels_val,
                                 cv=True
                                )

        run_time = time.time() - start_time


        # --------------------------------------------------------- #
        # ------------------- Save calculations ------------------- #
        # --------------------------------------------------------- #
        save_data(time_now,
                  folder_name,
                  run,
                  weights,
                  bias,
                  losses,
                  cv_j_train,
                  j_val,
                  pred_train_arr,
                  pred_val_arr,
                  acc_train_arr,
                  acc_val_arr,
                  run_time,
                  cv=True
                 )

Max train / Last run
---------------------------------------------------
  CV   | Run |   Iter    |Acc train|Acc val| Time  
---------------------------------------------------


TracerArrayConversionError: The numpy.ndarray conversion method __array__() was called on traced array with shape int64[].
The error occurred while tracing the function pred at /tmp/ipykernel_3564963/2856908872.py:13 for jit. This value became a tracer due to JAX operations on these lines:

  operation a:i64[] = convert_element_type[new_dtype=int64 weak_type=True] b
    from line /tmp/ipykernel_3564963/2856908872.py:20 (pred)

  operation a:i64[] = convert_element_type[new_dtype=int64 weak_type=True] b
    from line /tmp/ipykernel_3564963/2856908872.py:20 (pred)

  operation a:i64[] = convert_element_type[new_dtype=int64 weak_type=False] b
    from line /tmp/ipykernel_3564963/2856908872.py:20 (pred)

  operation a:bool[] = lt b c
    from line /tmp/ipykernel_3564963/2856908872.py:20 (pred)

  operation a:i64[] = convert_element_type[new_dtype=int64 weak_type=False] b
    from line /tmp/ipykernel_3564963/2856908872.py:20 (pred)

(Additional originating lines are not shown.)
See https://

# Miscellaneous

In [ ]:
# print(mp.cpu_count()) #256

# gs = gs_train
# lab = labels_train

# for i in range(10, 120, 10):
#     num_cpus_test = i
#     start_time = time.time()
#     loss(weights[0], bias[0], gs, lab, num_cpus_test)
#     print(f"{i} --- {(time.time() - start_time):.2f} seconds ---")

In [ ]:
# define regions coordinates
x01, y01 = region01e_coords[:,0], region01e_coords[:,1]
x02, y02 = region02e_coords[:,0], region02e_coords[:,1]
x1, y1 = region1e_coords[:,0], region1e_coords[:,1]
x2, y2 = region2e_coords[:,0], region2e_coords[:,1]
x3, y3 = region3e_coords[:,0], region3e_coords[:,1]

# put the regions into the plot
plt.fill(x01, y01, facecolor='lightskyblue')    # class 0
plt.fill(x02, y02, facecolor='lightskyblue')    # class 0
plt.fill(x1, y1, facecolor='sandybrown')        # class 1
plt.fill(x2, y2, facecolor='salmon')            # class 2
plt.fill(x3, y3, facecolor='lightgreen')        # class 3

plt.axis('square')
plt.show()